In [111]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
DESCRIPTION

Compute MODIS cloud fraction for one year.

"""

# Import modules
import numpy as np
import pandas as pd
import glob
import os
import netCDF4
import pyresample
from scipy.interpolate import griddata
from pyhdf.SD import SD, SDC

In [112]:
# Define a function
def sds_read(dataset, variable):
    
    # Read SDS
    sds = dataset.select(variable) 
    sds_scale = sds.attributes()['scale_factor']
    sds_offset = sds.attributes()['add_offset']
    
    mask = (sds.get() == sds.attributes()['_FillValue'])
    sds_float = sds.get().astype('float')
    sds_float[mask] = np.nan
    
    return (sds_float - sds_offset) * sds_scale

    
def MYD06_L2_Read(myd, attribute):
    
    # Read MODIS file
    f = SD(myd, SDC.READ)
    
    # Get datasets
    sds_lat = f.select('Latitude')
    latitude = sds_lat.get()
    
    sds_lon = f.select('Longitude')
    longitude = sds_lon.get()
    
    att = sds_read(f, attribute)
    
    # Interpolate some attributes from 5 km to 1 km    
    grid_x_1km, grid_y_1km = np.meshgrid(np.linspace(0, latitude.shape[1]-1, att.shape[1]), 
                                 np.linspace(0, latitude.shape[0]-1, att.shape[0]))
    
    grid_x_5km, grid_y_5km = np.meshgrid(np.arange(0, latitude.shape[1], 1), 
                                 np.arange(0, latitude.shape[0], 1))
    
    latitude_1km = griddata((np.ravel(grid_x_5km), np.ravel(grid_y_5km)), 
                       np.ravel(latitude), (grid_x_1km, grid_y_1km), 
                       method='linear')
    longitude_1km = griddata((np.ravel(grid_x_5km), np.ravel(grid_y_5km)), 
                       np.ravel(longitude), (grid_x_1km, grid_y_1km), 
                       method='linear')
    
    data = np.dstack((latitude_1km[50:-50, 50:-50], longitude_1km[50:-50, 50:-50], 
                  att[50:-50, 50:-50]))

    return data

In [113]:
# Define path
path = '/Users/jryan4/Dropbox (University of Oregon)/research/clouds/data/'

# Define files
modis_list = sorted(glob.glob('/Volumes/Extreme Pro/clouds/*.hdf'))

# Define destination for predicted data
dest = path + 'modis_cloud_properties/'

# Define ice sheet grid
ismip = netCDF4.Dataset(path + 'masks/1km-ISMIP6.nc')
ismip_lon = ismip.variables['lon'][:]
ismip_lat = ismip.variables['lat'][:]

# Define years
years = np.arange(2003, 2021, 1)

# Define good hours
good_hours = ['06', '07', '08', '09', '10', '11', '12', '13', '14']

In [114]:
good_files = []
for file in modis_list:
    # Get path and filename seperately 
    infilepath, infilename = os.path.split(file)
    # Get file name without extension            
    infilehortname, extension = os.path.splitext(infilename)
    
    # Append hour
    hour = infilehortname[18:20]
    if hour in good_hours:
        good_files.append(file)

In [ ]:
# Define number of files to chunk
number = 50

# Chunk into groups of 50
bounds = np.arange(0, len(good_files), number)

for bound in bounds:

    # Get slice of list
    files_sliced = good_files[bound:bound+number]

    data_stacked = np.zeros((2881, 1681))
    for i in range(len(files_sliced)):
        print('Processing... %.0f out of %.0f' %(i+1, len(files_sliced)))

        # Read HDF
        data = MYD06_L2_Read(files_sliced[i], 'Cloud_Phase_Optical_Properties')

        # Set zeros to NaNs
        data[:,:,2][data[:,:,2] == 0] = np.nan

        # Convert liquid , ice, and undetermined flags to 2
        data[:,:,2][data[:,:,2] > 1] = 2

        # 2 = CTH, 3 = CTT, 4 = CTP, 5 = PHASE, 6 = COT, 7 = CER, 8 = CWP
        if np.nansum(np.isfinite(data[:,:,2])) > 200000:
            # Resample radiative fluxes to ISMIP grid
            swath_def = pyresample.geometry.SwathDefinition(lons=data[:,:,1], lats=data[:,:,0])
            swath_con = pyresample.geometry.GridDefinition(lons=ismip_lon, lats=ismip_lat)


            # Determine nearest (w.r.t. great circle distance) neighbour in the grid.
            data_resampled = pyresample.kd_tree.resample_nearest(source_geo_def=swath_def, 
                                                             target_geo_def=swath_con, 
                                                             data=data[:,:,2],
                                                             radius_of_influence=5000)

            # Set zeros to NaNs
            data_resampled[data_resampled == 0] = np.nan

            # Stack
            data_stacked = np.dstack((data_stacked, data_resampled))

    # Remove first layer
    data_stacked = data_stacked[:, :, 1:]

    # Convert values
    data_stacked[data_stacked == 1] = 0
    data_stacked[data_stacked == 2] = 1

    # Average
    data_mean = np.nanmean(data_stacked, axis=2)

    ###############################################################################
    # Save 1 km dataset to NetCDF
    ###############################################################################
    dataset = netCDF4.Dataset(dest + 'myd06_cloud_' + str(bound) + '_' + str(bound+number) + '.nc', 
                              'w', format='NETCDF4_CLASSIC')
    print('Creating... %s' % dest + 'myd06_cloud_' + str(bound) + '_' + str(bound+number)  + '.nc')
    dataset.Title = "Cloud fraction"
    import time
    dataset.History = "Created " + time.ctime(time.time())
    dataset.Projection = "WGS 84"
    dataset.Reference = "Ryan, J. C., Smith, L. C., et al. (unpublished)"
    dataset.Contact = "jryan4@uoregon.edu"

    # Create new dimensions
    lat_dim = dataset.createDimension('y', ismip_lat.shape[0])
    lon_dim = dataset.createDimension('x', ismip_lat.shape[1])

    # Define variable types
    Y = dataset.createVariable('latitude', np.float32, ('y','x'))
    X = dataset.createVariable('longitude', np.float32, ('y','x'))

    y = dataset.createVariable('y', np.float32, ('y'))
    x = dataset.createVariable('x', np.float32, ('x'))


    # Define units
    Y.units = "degrees"
    X.units = "degrees"

    # Create the actual 3D variable
    cloud_fraction_nc = dataset.createVariable('cloud_fraction', np.float32, ('y','x'))

    # Write data to layers
    Y[:] = ismip_lat
    X[:] = ismip_lon
    x[:] = ismip_lon[0,:]
    y[:] = ismip_lat[:,0]
    cloud_fraction_nc[:] = data_mean

    print('Writing data to %s' % dest + 'myd06_cloud_' + str(bound) + '_' + str(bound+number) + '.nc')

    # Close dataset
    dataset.close()